In [1]:
# 9 - 20 - Function overriding with different types 
# 1 Child class to Parent class (upcast) is unconditional
class Foo:
    def __init__(self, a):
        self.a = a 
class Foo2(Foo):
    def __init__(self, a, b):
        super().__init__(a)
        self.b = b
f2 = Foo2(1,2)
print(vars(f2))
print(isinstance(f2, Foo))

# Failed attempt 1
# class -> meta that has MultiDict as clsdict. ->
# MultiDict will store {(type, func_name): method}
class MultiDict(dict):
     def __setitem__(self, key , value ) -> None:
         # We must store string, not tuples because when class gets created, 
         # names in string are compared. But the modified names are 
         # already stored in the class.
         # Failure Reason: therefore, we must store the same key 
         new_key = str((key, 1))
         print(new_key, "val: ", value)
         super().__setitem__(new_key, value)
     def __getitem__(self, key):
        print("get_item, key: ", key)
        return super().__getitem__((key, 1)) 

class Meta(type):
    @classmethod
    def __prepare__(cls, clsname, bases):
        return MultiDict()

    # def __new__(cls, clsname, bases, clsdict):
    #     return 
class Spam(metaclass = Meta):
    def s(self, a):
        self.a = a
sp = Spam()
try: 
    sp.s(1)
except AttributeError:
    pass




{'a': 1, 'b': 2}
True
get_item, key:  __name__
('__module__', 1) val:  __main__
('__qualname__', 1) val:  Spam
('s', 1) val:  <function Spam.s at 0x7f71de99ddc0>


In [105]:
# 9 - 20 - CROWN JEWEL OF CHAP 9: Function overriding with different types 
# inspect.signature(func)->parameters->name, annotation
import inspect
class TypeMethodRegistry:
    def __init__(self):
        # registry to store different functions under the same name based on their arg list 
        self.registry = {}
        
    def register(self, method):
        '''
        Function takes care of storing functions based on their types
        '''
        # signature has parameters, you have to get their annotations manually
        func_type = []
        # we don't want self, because in signature, the first annotation's type is Meta, 
        # which does not have __eq__ for comparison
        for param in inspect.signature(method).parameters.values():
            if param.name not in ("cls", "self"):
                func_type.append(param.annotation)
        print("registered type: ", func_type)
        # need to hash the list
        self.registry[tuple(func_type)] = method
        
    def __call__(self, *args, **kwds):
        '''
        Function returns the right function based on its types
        self is TypeMethodRegistry, but args will contain the actual instance (Spam)
        '''
        # args will be the values of the arguments directly, including self (which has been bound)
        # Not supporting kwargs, because we need the ORDER of arguments
        arg_types = []
        arg_types = tuple([type(arg) for arg in args[1:]])

        if arg_types in self.registry.keys(): 
            print("Spam is in the args since we have bound this TypeMethodRegistry object to it in __get__: ", args)
            print("So we can pass Spam, and other args to the actual function")
            return self.registry[arg_types](*args)
        else:
            raise AttributeError(f"Function argument annotations {arg_types} are not found in functions with the same name", 
                                 f" {self.registry.keys()}")
        

    def __get__(self, instance, cls):
        '''
        When we call a function of an instance/ class, we call the function's __get__. 
        Then we bound the function to the instance
        In this case, we are using MultiDict, which directly returns TypeMethodRegistry object,
        Which further returns the right function. We need to bind the TypeMethodRegistry to the right 
        instance 
        '''
        
        # cls is always not None
        if instance is None:
            return self
        else:
            from types import MethodType
            print(f"self: {type(self)}, instance: {type(instance)}")
            return MethodType(self, instance)
        
        
class MultiDict(dict):
    '''
    Dict-like object that __prepare__() returns.
    keys are module/ function names
    values are strings (module) or methods
    '''
    # Overriding dictionary method.
    def __setitem__(self, method_name, method_or_str) -> None:
        '''
        We check if we already have the method / module name
        [TAKEAWAY]: You can use try, but in Python dict.__setitem__, you can check by 
        if method in self.
        '''
        if method_name in self:
        # try:
            registry = super().__getitem__(method_name)
            registry.register(method_or_str)
        else:
        # except KeyError as e:
            # because the dict __prepare__ receives things like {"module": "__main__"}        
            if callable(method_or_str):
                method = method_or_str
                registry = TypeMethodRegistry()
                registry.register(method)
            else:
                registry = method_or_str
            super().__setitem__(method_name, registry)
        
    
class Meta(type):
    '''
    Meta's {"func_name": TypeMethodRegistry([method1, method2 ...])} -> Instances (Spam)
    '''
    @classmethod
    def __prepare__(cls, clsname, bases):
        # Have to return a dictionary, with (key, val)
        # In this case, key is function name, val is the method itself 
        return MultiDict()
    
    
class Spam(metaclass = Meta):
    def s(self: Spam):
        print("s1")
    def s(self: Spam, a:int):
        self.a = a
        print("hello")
    def s(self, a:int, b: float):
        self.a = a
        print(self.s.__name__)
sp = Spam()
sp.s()

# So now s() is a class variable, which calls __get__(instance, cls) (Descriptor Protocal for all class variables)
# To get a bound function 
sp.s(1)


registered type:  []
registered type:  [<class 'int'>]
registered type:  [<class 'int'>, <class 'float'>]
self: <class '__main__.TypeMethodRegistry'>, instance: <class '__main__.Spam'>
Spam is in the args since we have bound this TypeMethodRegistry object to it in __get__:  (<__main__.Spam object at 0x7f71dcf21b50>,)
So we can pass Spam, and other args to the actual function
s1
self: <class '__main__.TypeMethodRegistry'>, instance: <class '__main__.Spam'>
Spam is in the args since we have bound this TypeMethodRegistry object to it in __get__:  (<__main__.Spam object at 0x7f71dcf21b50>, 1)
So we can pass Spam, and other args to the actual function
hello


In [34]:
# 9 - 20 - 3 
# Function Overloading using decorator 
# Key Technique: use descriptor as a decorator. 
# See advanced_oop.py -> FooDescriptor for usage
import inspect
class multidictdescriptor:
    def __init__(self, default_func):
        # Step 1 - In descriptor, just need __init__ to store the function
        # Then multidictdescriptor instance replaces the function, as a proxy
        # So later, in the class, we can call f.match
        self.default_func = default_func
        self.registry = {}
         
    def __call__(self, *args, **kwargs):
        # Step 3 - now multidictdescriptor as a proxy, 
        # we need to return the real function
        args_type = [type(arg) for arg in args]
        args_key = tuple(args_type[1:])
        print("call: args key", args_key, " registry: ", self.registry)
        if  args_key in self.registry.keys():
            return self.registry[args_key](*args)
        else:
            return self.default_func(*args, **kwargs)
    
    def match(self, *args):
        # pass types in, like multidictdescriptor.match(int, int)
        def register(func):
            print("function defaults: ", func.__defaults__)
            # TODO: add a check to check if the func signature matches the args
            # since args are passed directly from the decorator,
            # There's no self
            self.registry[args] = func
            # want to return the same multidictdescriptor instance itself back to Spam
            return self
        # the outer layer of decorator always returns the inner decorator function
        return register
        
    def __get__(self, instance, cls):
        # Step 2 - When we call multidictdescriptor instance, 
        # first step is to bind it with Spam instance, 
        # or just return multidictdescriptor instance itself as class instance
        if instance is None:
            return self
        else:
            from types import MethodType
            return MethodType(self, instance)
class Spam:
    @multidictdescriptor
    def f(self, *args, **kwargs):
        raise TypeError(f"No function implemented for args {args}, kwargs {kwargs}")
    
    @f.match(int, int)
    def f(self, a, b):
        print("f1")
    # def f(self, a):
    #     print("f2, ", a)

s = Spam()
s.f(1,2)


None
call: args key (<class 'int'>, <class 'int'>)  registry:  {(<class 'int'>, <class 'int'>): <function Spam.f at 0x7fcdbd6df670>}
f1
